In [4]:
import numpy as np
import torch
from d2l import torch as d2l

In [5]:
X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)

torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 0.1380, -0.5316,  0.7386,  1.5640],
        [-0.3534,  0.4705,  1.6298, -1.0641],
        [ 1.0711,  0.6479, -0.0846,  0.4029]])

In [6]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[ 0.1380, -0.5316,  0.7386,  1.5640],
        [-0.3534,  0.4705,  1.6298, -1.0641],
        [ 1.0711,  0.6479, -0.0846,  0.4029]])